In [1]:
from dotenv import load_dotenv
load_dotenv(dotenv_path="../../.env", override=True)

False

In [3]:
from langsmith import traceable
from openai import OpenAI
from typing import List
import nest_asyncio
from utils import get_vector_db_retriever

MODEL_PROVIDER = "openai"
MODEL_NAME = "gpt-4o-mini"
APP_VERSION = 1.0
RAG_SYSTEM_PROMPT = """You are an assistant for question-answering tasks. 
Use the following pieces of retrieved context to answer the latest question in the conversation. 
If you don't know the answer, just say that you don't know. 
Use three sentences maximum and keep the answer concise.
"""

openai_client = OpenAI()
nest_asyncio.apply()
retriever = get_vector_db_retriever()

# TODO: Set up tracing for each function
@traceable
def retrieve_documents(question: str):
    return retriever.invoke(question)   # NOTE: This is a LangChain vector db retriever, so this .invoke() call will be traced automatically

@traceable
def generate_response(question: str, documents):
    formatted_docs = "\n\n".join(doc.page_content for doc in documents)
    messages = [
        {
            "role": "system",
            "content": RAG_SYSTEM_PROMPT
        },
        {
            "role": "user",
            "content": f"Context: {formatted_docs} \n\n Question: {question}"
        }
    ]
    return call_openai(messages)

@traceable
def call_openai(
    messages: List[dict], model: str = MODEL_NAME, temperature: float = 0.0
) -> str:
    return openai_client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature,
    )

@traceable
def langsmith_rag(question: str):
    documents = retrieve_documents(question)
    response = generate_response(question, documents)
    return response.choices[0].message.content

In [4]:
question = "How can I trace with the @traceable decorator?"
ai_answer = langsmith_rag(question)
print(ai_answer)

To trace with the @traceable decorator in Python, simply decorate any function you want to log traces for with @traceable. Ensure that the LANGSMITH_TRACING environment variable is set to 'true' and the LANGSMITH_API_KEY is configured with your API key. When calling a synchronous function wrapped with @traceable, use the await keyword to ensure the trace is logged correctly.


In [5]:
question = "What is roonaldo famous for?"
ai_answer = langsmith_rag(question)
print(ai_answer)

I don't know.


In [6]:
question = "How does LangSmith handle nested function calls and maintain the trace hierarchy?"
ai_answer = langsmith_rag(question)
print(ai_answer)

LangSmith automatically propagates the correct context for tracing, ensuring that code executed within a parent trace is rendered in the expected location in the UI. However, if a child run appears in a separate trace, it may be due to known edge cases, particularly in Python. Proper context propagation is crucial for maintaining the trace hierarchy during nested function calls.


In [7]:
from langsmith import traceable

@traceable(
    # TODO: Add Metadata
    metadata={"vectordb": "sklearn"}
)
def retrieve_documents(question: str):
    return retriever.invoke(question)

@traceable
def generate_response(question: str, documents):
    formatted_docs = "\n\n".join(doc.page_content for doc in documents)
    messages = [
        {
            "role": "system",
            "content": RAG_SYSTEM_PROMPT
        },
        {
            "role": "user",
            "content": f"Context: {formatted_docs} \n\n Question: {question}"
        }
    ]
    return call_openai(messages)

@traceable(
    # TODO: Add Metadata
    metadata={"gpt-4o-mini": MODEL_NAME, "openai": MODEL_PROVIDER}
)
def call_openai(
    messages: List[dict], model: str = MODEL_NAME, temperature: float = 0.0
) -> str:
    return openai_client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature,
    )

@traceable
def langsmith_rag(question: str):
    documents = retrieve_documents(question)
    response = generate_response(question, documents)
    return response.choices[0].message.content

In [8]:
question = "How do I add Metadata to a Run with @traceable?"
ai_answer = langsmith_rag(question)
print(ai_answer)

You can add metadata to a run by including a metadata key in the `experimental_telemetry` object when calling the `generateText` function. For example, you can pass metadata like `{ userId: "123", language: "english" }` within the `experimental_telemetry` parameter. This metadata will then be visible in your LangSmith dashboard and can be used for filtering and searching traces.


In [9]:
question = "How do I add metadata at runtime?"
ai_answer = langsmith_rag(question, langsmith_extra={"metadata": {"runtime_metadata": "foo"}})
print(ai_answer)

To add metadata at runtime, you can attach it when running an experiment in the SDK. This involves specifying the metadata related to your experiment, such as the model used or other relevant identifiers. Ensure that the same metadata is also attached to the trace for consistent filtering and analysis later.


In [15]:
question = "what is  the current president of india?"
ai_answer = langsmith_rag(question, langsmith_extra={"metadata": {"runtime_metadata": "foo"}})
print(ai_answer)

I don't know.


In [17]:
question = "What's the minimum setup needed for production tracing?"
ai_answer = langsmith_rag(question, langsmith_extra={"metadata": {"runtime_metadata": "foo"}})
print(ai_answer)

The minimum setup needed for production tracing includes configuring the following environment variables: `LANGSMITH_TRACING`, `LANGSMITH_API_KEY`, `LANGSMITH_ENDPOINT`, and `LANGSMITH_PROJECT`. Additionally, it's important to set limits on production usage based on the expected load of traces. This ensures effective management of tracing costs and data retention.
